In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [28]:
# CSVファイルの読み込み (伝吉くんの整備してくれたデータ)
df = pd.read_csv("data4/20250306features.csv")


In [29]:
df.describe()

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,umaban,wakuban,tansho_odds,popularity,...,place,month,sin_date,cos_date,horse_potential1,horse_potential2,horse_potential_iv,jockey_potential1,jockey_potential2,jockey_potential_iv
count,3.383760e+05,3.383760e+05,338376.000000,338376.000000,338376.000000,338376.000000,338376.000000,338376.000000,338376.000000,338376.000000,...,338376.000000,338376.000000,3.383760e+05,338376.000000,12966.000000,12966.000000,12966.000000,12966.000000,12966.000000,12966.000000
mean,2.021130e+11,2.017544e+09,1450.969761,1028.925166,479426.069003,7.652363,7.707795,4.761003,67.220530,7.678296,...,6.179339,6.295642,1.036936e+00,1.025713,0.253352,0.447852,0.474281,0.685464,0.440969,0.196779
std,2.062609e+08,2.463407e+06,1214.972859,280.413292,295186.644288,4.376705,4.401496,2.271146,99.906394,4.388136,...,2.432284,3.501973,7.040979e-01,0.708677,1.173700,0.160871,0.172999,1.308434,0.204022,0.184805
min,2.018010e+11,2.006107e+09,409.000000,261.000000,31.000000,1.000000,1.000000,1.000000,1.100000,1.000000,...,1.000000,1.000000,5.779753e-07,0.000021,-3.619323,0.000000,0.000000,-2.341618,0.000000,0.000000
25%,2.019081e+11,2.016101e+09,1077.000000,1039.000000,226800.000000,4.000000,4.000000,3.000000,8.100000,4.000000,...,5.000000,3.000000,3.227601e-01,0.326725,-0.590670,0.335832,0.351976,-0.173061,0.298042,0.073270
50%,2.021061e+11,2.017110e+09,1138.000000,1090.000000,486800.000000,7.000000,7.000000,5.000000,24.500000,7.000000,...,6.000000,6.000000,1.103031e+00,1.086977,0.201906,0.439594,0.480018,0.469649,0.418305,0.127271
75%,2.023050e+11,2.019106e+09,1174.000000,1137.000000,758005.000000,11.000000,11.000000,7.000000,80.500000,11.000000,...,8.000000,9.000000,1.735034e+00,1.726964,0.998435,0.551447,0.606175,1.425866,0.569351,0.253107
max,2.025100e+11,2.022190e+09,5673.000000,5785.000000,999031.000000,18.000000,18.000000,8.000000,999.900000,18.000000,...,10.000000,12.000000,1.999986e+00,1.999251,4.534233,1.000000,1.000000,4.725002,1.000000,1.000000


In [30]:

# race_idでグループ化し、Nを集計
d1 = df.groupby('race_id').size().reset_index(name='N')
d1 = d1[d1['N'] > 9]

# race_idがd1のrace_idに含まれる行をフィルタリング
df2 = df[df['race_id'].isin(d1['race_id'])]

# 再度race_idでグループ化し、Nを集計
d2 = df2.groupby('race_id').size().reset_index(name='N')

# 出走頭数別頻度を表示
print(d2['N'].value_counts())

N
16    7247
15    3119
14    2513
12    1996
13    1984
11    1789
10    1495
18    1487
17     532
Name: count, dtype: int64


In [31]:
# date列をdatetime型に変換
df2['date'] = pd.to_datetime(df2['date'])

/tmp/ipykernel_23082/3038544232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['date'] = pd.to_datetime(df2['date'])


In [32]:
#読み込んだデータの日付の範囲を表示
print(df2['date'].min())
print(df2['date'].max())

2018-01-06 00:00:00
2025-02-23 00:00:00


In [33]:
# 列名を表示する
df2.columns

Index(['race_id', 'date', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id',
       'rank', 'umaban', 'wakuban', 'tansho_odds', 'popularity', 'impost',
       'sex', 'age', 'weight', 'weight_diff', 'n_horses', 'time', 'time_rank',
       '着差', 'race_type', 'around', 'course_len', 'weather', 'ground_state',
       'race_class', 'place', 'month', 'sin_date', 'cos_date',
       'horse_potential1', 'horse_potential2', 'horse_potential_iv',
       'jockey_potential1', 'jockey_potential2', 'jockey_potential_iv'],
      dtype='object')

In [34]:
#能力値の列は一旦削除する
df2 = df2.drop(['horse_potential1', 'horse_potential2', 'horse_potential_iv',
       'jockey_potential1', 'jockey_potential2', 'jockey_potential_iv'
        ], axis=1)       

In [35]:
# race_classでグループ化し、 データ数を集計
d2 = df2.groupby('race_class').size().reset_index(name='N')
print(d2)

    race_class       N
0           -4  119527
1           -3    1409
2           -2     942
3           -1    3077
4            0   20875
5            1   87546
6            2   13436
7            3    5275
8            4   17249
9            5   31993
10           6    4524
11           7    6593
12           8    3057
13           9    2599


In [36]:
# Num列を追加
df2.insert(0, 'Num', range(1, len(df2) + 1))

In [37]:

# Num列を降順に並べ替え
df3 = df2.sort_values(by='Num', ascending=False)

In [38]:
len(df3['race_id'].unique())

22162

In [39]:
# ラベルエンコーディングの対象となる列を指定
label_cols = ['race_id', 'horse_id', 'jockey_id', 'trainer_id', 'owner_id']
# LabelEncoderによる変換
label_encoder = LabelEncoder()

# データの変換
for col in label_cols:
    df3[col] = label_encoder.fit_transform(df3[col])
    



In [40]:
table1 = df2.loc[:,['Num']+label_cols]
table2 = df3.loc[:,['Num']+label_cols]
map_table = table1.merge(table2, on='Num', how='left')

In [41]:
# 重複行を削除
mapping_table = map_table.drop_duplicates()
# CSVファイルに保存
mapping_table.to_csv("data4/python/mapping_table.csv", index=False)

In [ ]:
def most_frequent(x):
    freq_table = pd.Series(x).value_counts()
    most = freq_table.idxmax()
    return most

In [ ]:
#新特徴量を追加
df3['impost_weight'] = df3['impost'] / df3['weight']
df3['weight_diff_prop'] = df3['weight_diff'] / df3['weight']
df3['great_odds'] = (df3['popularity'] - df3['rank']) / df3['n_horses']
df3['speed'] = df3['course_len'] / df3['time']
df3['odds_rate'] = df3['tansho_odds'] / (df3['tansho_odds'] + 1)
df3['rate'] = 1 / df3['n_horses']
df3['rate_ratio'] = df3['odds_rate'] / df3['rate']

In [ ]:
from tqdm import tqdm

# 過去の成績の参照用データフレームの作成
record = pd.DataFrame({
    'N_id': [np.nan],
    'exp_num': [np.nan],
    'jockeyID_past_frq': [np.nan],
    'rank_past_mea': [np.nan],
    'type_rank_past_mea': [np.nan],
    'corse_len_rank_past_mea': [np.nan],
    'rank_past_var': [np.nan],
    'type_rank_past_var': [np.nan],
    'corse_len_rank_past_var': [np.nan],
    'odds_past_mea': [np.nan],
    'odds_past_var': [np.nan],
    'popularity_past_mea': [np.nan],
    'impost_weight_past_mea': [np.nan],
    'impost_weight_past_var': [np.nan],
    'weight_diff_prop_past_mea': [np.nan],
    'weight_diff_prop_past_var': [np.nan],
    'time_past_mea': [np.nan],
    'speed_past_mea': [np.nan],
    'race_class_past_frq': [np.nan],
    'place_past_frq': [np.nan],
    'great_odds_past_mea': [np.nan],
    'great_odds_past_var': [np.nan],
    'interval_diff': [np.nan],
    'rate_rario_past_mea': [np.nan],
    'rate_rario_past_var': [np.nan]
})

i = 1
for t in tqdm(range(len(df3))):
    df3_1 = df3.iloc[t]
    horse_ori = df3_1['horse_id']
    date_ori = df3_1['date']
    place_ori = df3_1['place']
    course_ori = df3_1['course_len']
    racetype_ori = df3_1['race_type']
    N_ori = df3_1['Num']
    
    df4 = df3[df3['date'] < date_ori]
    df4_1 = df4[df4['horse_id'] == horse_ori]
    exp = len(df4_1) #過去の出走回数
    
    if len(df4_1) > 10: #過去10回の成績を参照
        df4_1 = df4_1.iloc[:10]
    
    df4_2 = df4[(df4['horse_id'] == horse_ori) & 
                (df4['race_type'] == racetype_ori) & 
                (df4['course_len'] == course_ori)]
    
    if len(df4_2) > 5:#過去5回の成績を参照（条件が厳しい）
        df4_2 = df4_2.iloc[:5]
    
    df4_3 = df4[(df4['horse_id'] == horse_ori) & 
                (df4['race_type'] == racetype_ori) ]
    
    if len(df4_3) > 5:#過去5回の成績を参照（条件が厳しい）
        df4_3 = df4_3.iloc[:5]
    
    hantei = len(df4_1)
    
    if hantei > 0:
        record.loc[i, 'N_id'] = N_ori
        record.loc[i, 'exp_num'] = exp
        record.loc[i, 'jockeyID_past_frq'] = most_frequent(df4_1['jockey_id'])
        record.loc[i, 'rank_past_mea'] = df4_1['rank'].mean()
        record.loc[i, 'type_rank_past_mea'] = df4_3['rank'].mean()
        record.loc[i, 'corse_len_rank_past_mea'] = df4_2['rank'].mean()
        record.loc[i, 'rank_past_var'] = df4_1['rank'].var()
        record.loc[i, 'type_rank_past_var'] = df4_3['rank'].var()
        record.loc[i, 'corse_len_rank_past_var'] = df4_2['rank'].var()
        record.loc[i, 'odds_past_mea'] = df4_1['tansho_odds'].mean()
        record.loc[i, 'odds_past_var'] = df4_1['tansho_odds'].var()
        record.loc[i, 'popularity_past_mea'] = df4_1['popularity'].mean()
        record.loc[i, 'impost_weight_past_mea'] = df4_1['impost_weight'].mean()
        record.loc[i, 'impost_weight_past_var'] = df4_1['impost_weight'].var()
        record.loc[i, 'weight_diff_prop_past_mea'] = df4_1['weight_diff_prop'].mean()
        record.loc[i, 'weight_diff_prop_past_var'] = df4_1['weight_diff_prop'].var()
        record.loc[i, 'time_past_mea'] = df4_2['time'].mean()
        record.loc[i, 'speed_past_mea'] = df4_2['speed'].mean()
        record.loc[i, 'race_class_past_frq'] = most_frequent(df4_1['race_class'])
        record.loc[i, 'place_past_frq'] = most_frequent(df4_1['place'])
        record.loc[i, 'great_odds_past_mea'] = df4_1['great_odds'].mean()
        record.loc[i, 'great_odds_past_var'] = df4_1['great_odds'].var()
        if len(df4_1) > 1:
            record.loc[i, 'interval_diff'] = (date_ori - df4_1['date'].iloc[1]).days
        else:
            record.loc[i, 'interval_diff'] = np.nan
        record.loc[i, 'rate_rario_past_mea'] = df4_1['rate_ratio'].mean()
        record.loc[i, 'rate_rario_past_var'] = df4_1['rate_ratio'].var()
        
        i += 1

In [ ]:
# df3とrecordをNum列とN_id列で結合
df6 = df3.merge(record, left_on='Num', right_on='N_id', how='left')
# race_classでグループ化し、欠測値の数と欠測率を計算
result = df6.groupby('race_class').apply(lambda x: pd.Series({
    'noting': x['exp_num'].isna().sum(),
    'rate': x['exp_num'].isna().sum() / len(x)
})).reset_index()

print(result)

In [ ]:
# df6をCSVファイルに保存
df6.to_csv("data4/python/0306features_with_past0328.csv", index=False)

過去成績を特徴量に加えたファイルの名前に_pastをつける